Lectura de xml

In [2]:
import pandas as pd
import xml.etree.ElementTree as etree



#tree = etree.parse("tg-fal17.xml")
tree = etree.parse("agh-ggis-spr17.xml")
root = tree.getroot()

columns_problem = ["Name", "NrDays", "SlotsPerDay", "NrWeeks"]
columns_optimization = ["Time", "Room", "Distribution", "Student"]
columns_rooms = ["RoomId", "Capacity", "PreviousRoomIdList", "TravelTimeValueList", "UnavailableDaysList", "UnavailableStartList", "UnavailableLengthList", "UnavailableWeeksList"]
columns_courses = ["CourseId", "ConfigIdList"]
columns_configs = ["ConfigId", "SubpartIdList"]
columns_subparts = ["SubpartId", "ClassIdList"]
columns_classes = ["ClassId", "ClassLimit", "ClassParent"]
columns_distributions = ["DistributionId", "DistributionType", "DistributionRequired", "DistributionPenalty", "ClassesList"]
columns_students = ["StudentId", "CoursesList"]
columns_relation_class_room = ["ClassId", "RoomId", "RoomPenalty"]
columns_relation_class_time = ["ClassId", "TimeDays", "TimeStart", "TimeLength", "TimeWeeks", "TimePenalty"]

df_problem = pd.DataFrame(columns = columns_problem)
df_optimization = pd.DataFrame(columns = columns_optimization)
df_rooms = pd.DataFrame(columns = columns_rooms)
df_courses = pd.DataFrame(columns = columns_courses)
df_configs = pd.DataFrame(columns = columns_configs)
df_subparts = pd.DataFrame(columns = columns_subparts)
df_classes = pd.DataFrame(columns = columns_classes)
df_distributions = pd.DataFrame(columns = columns_distributions)
df_students = pd.DataFrame(columns = columns_students)
df_relation_class_room = pd.DataFrame(columns = columns_relation_class_room)
df_relation_class_time = pd.DataFrame(columns = columns_relation_class_time)




df_problem = df_problem.append({"Name": root.attrib.get("name"), "NrDays": root.attrib.get("nrDays"), "SlotsPerDay": root.attrib.get("slotsPerDay"), "NrWeeks": root.attrib.get("nrWeeks")}, ignore_index = True)

for node in root:
    if node.tag == "optimization":
        df_optimization = df_optimization.append({"Time": node.attrib.get("time"), "Room": node.attrib.get("room"), "Distribution": node.attrib.get("distribution"), "Student": node.attrib.get("student")}, ignore_index = True)
    
    elif node.tag == "rooms":
        for room in node:
            roomid = room.attrib.get("id")
            cap = room.attrib.get("capacity")
            prevroomlist = []
            valuelist = []
            dayslist = []
            startlist = []
            lengthlist = []
            weekslist = []
            if len(list(child.tag for child in room.iter() if child is not room)) > 0:
                for child in room:
                    if child.tag == "travel":
                        prevroomid = child.attrib.get("room")
                        value = child.attrib.get("value")
                        prevroomlist.append(prevroomid)
                        valuelist.append(value)
                    elif child.tag == "unavailable":
                        days = child.attrib.get("days")
                        start = child.attrib.get("start")
                        length = child.attrib.get("length")
                        weeks = child.attrib.get("weeks")
                        dayslist.append(days)
                        startlist.append(start)
                        lengthlist.append(length)
                        weekslist.append(weeks)
            if len(prevroomlist) == 0:
                prevroomlist = None
            if len(valuelist) == 0:
                valuelist = None
            if len(dayslist) == 0:
                dayslist = None
            if len(startlist) == 0:
                startlist = None
            if len(lengthlist) == 0:
                lengthlist = None
            if len(weekslist) == 0:
                weekslist = None
            df_rooms = df_rooms.append(pd.Series([roomid, cap, prevroomlist, valuelist, dayslist, startlist, lengthlist, weekslist], index = columns_rooms), ignore_index = True)
            
    elif node.tag == "courses":
        for course in node:
            #print(course.tag)
            courseid = course.attrib.get("id")
            configlist = []
            for config in course:
                #print(config.tag)
                configid = config.attrib.get("id")
                configlist.append(configid)
                subpartlist = []
                for subpart in config:
                    #print(subpart.tag)
                    subpartid = subpart.attrib.get("id")
                    subpartlist.append(subpartid)
                    claselist = []
                    for clase in subpart:
                        #print(clase.tag)
                        claseid = clase.attrib.get("id")
                        claselist.append(claseid)
                        claselimit = clase.attrib.get("limit")
                        claseparent = clase.attrib.get("parent") if clase.attrib.get("parent") is not None else None
                        df_classes = df_classes.append(pd.Series([claseid, claselimit, claseparent], index = columns_classes), ignore_index = True)
                        if len(list(child.tag for child in clase.iter() if child is not clase)) > 0:
                            for child in clase:
                                #print(child.tag)
                                if child.tag == "room":
                                    roomid = child.attrib.get("id")
                                    roompenalty = child.attrib.get("penalty")
                                    df_relation_class_room = df_relation_class_room.append(pd.Series([claseid, roomid, roompenalty], index = columns_relation_class_room), ignore_index = True)
                                elif child.tag == "time":
                                    timedays = child.attrib.get("days")
                                    timestart = child.attrib.get("start")
                                    timelength = child.attrib.get("length")
                                    timeweeks = child.attrib.get("weeks")
                                    timepenalty = child.attrib.get("penalty")
                                    df_relation_class_time = df_relation_class_time.append(pd.Series([claseid, timedays, timestart, timelength, timeweeks, timepenalty], index = columns_relation_class_time), ignore_index = True)
                    df_subparts = df_subparts.append(pd.Series([subpartid, claselist], index = columns_subparts), ignore_index = True)
                df_configs = df_configs.append(pd.Series([configid, subpartlist], index = columns_configs), ignore_index = True)
            df_courses = df_courses.append(pd.Series([courseid, configlist], index = columns_courses), ignore_index = True)
                                    
    elif node.tag == "distributions":
        contador = 1
        for distribution in node:
            #print(distribution.tag)
            distribid = contador
            distribtype = distribution.attrib.get("type")
            distribrequired = distribution.attrib.get("required") if distribution.attrib.get("required") is not None else None
            distribpenalty = distribution.attrib.get("penalty") if distribution.attrib.get("penalty") is not None else None
            listaclases = []
            for clase in distribution:
                #print(clase.tag)
                listaclases.append(clase.attrib.get("id"))
            df_distributions = df_distributions.append(pd.Series([distribid, distribtype, distribrequired, distribpenalty, listaclases], index = columns_distributions), ignore_index = True)
            contador += 1
             
    elif node.tag == "students":
        for student in node:
            studentid = student.attrib.get("id")
            listacourses = []
            for course in student:
                listacourses.append(course.attrib.get("id"))
            df_students = df_students.append(pd.Series([studentid, listacourses], index = columns_students), ignore_index = True)
            
            


print("################################################################################################################")
print("DataFrame Problem")
display(df_problem)
print("################################################################################################################")
print("DataFrame Optimization")
display(df_optimization)
print("################################################################################################################")
print("DataFrame Rooms")
display(df_rooms)
print("################################################################################################################")
print("DataFrame Courses")
display(df_courses)
print("################################################################################################################")
print("DataFrame Configs")
display(df_configs)
print("################################################################################################################")
print("DataFrame Subparts")
display(df_subparts)
print("################################################################################################################")
print("DataFrame Classes")
display(df_classes)
print("################################################################################################################")
print("DataFrame Distributions")
display(df_distributions)
print("################################################################################################################")
print("DataFrame Students")
display(df_students)
print("################################################################################################################")
print("DataFrame RelationClassRoom")
display(df_relation_class_room)
print("################################################################################################################")
print("DataFrame RelationClassTime")
display(df_relation_class_time)
print("################################################################################################################")

################################################################################################################
DataFrame Problem


,Name,NrDays,SlotsPerDay,NrWeeks
0,agh-ggis-spr17,7,288,16


################################################################################################################
DataFrame Optimization


,Time,Room,Distribution,Student
0,4,1,15,5


################################################################################################################
DataFrame Rooms


...

################################################################################################################
DataFrame RelationClassRoom


,ClassId,RoomId,RoomPenalty
0,1,43,0
1,2,43,0
2,3,43,0
3,4,43,0
4,5,43,0
...,...,...,...
12372,1833,32,0
12373,1835,24,0
12374,1838,24,0
12375,1839,24,0


################################################################################################################
DataFrame RelationClassTime


,ClassId,TimeDays,TimeStart,TimeLength,TimeWeeks,TimePenalty
0,1,1000000,96,18,0101010010010101,0
1,1,0100000,96,18,0101010010010101,0
2,1,0010000,96,18,0101010010010101,0
3,1,0000100,96,18,0101010010101010,0
4,1,1000000,106,18,0101010010010101,0
...,...,...,...,...,...,...
46662,1836,0100000,176,18,0101010010010101,0
46663,1837,0100000,156,18,1010101000101010,0
46664,1838,0100000,176,18,0101010010010101,0
46665,1839,0100000,156,18,1010101000101010,0


################################################################################################################


Lectura xml solucion

In [2]:
import pandas as pd
import xml.etree.ElementTree as etree

tree = etree.parse("solution-tg-fal17.xml")
root = tree.getroot()

columns_sol = ["ClassId", "Days", "Start", "Weeks", "Room"]

df_sol = pd.DataFrame(columns = columns_sol)

for clase in root:
    

SyntaxError: unexpected EOF while parsing (<ipython-input-2-2edd5ad0955c>, line 12)

Analisis de datos

In [44]:
## Duracion en slots de clases:

df_relation_class_time["TimeLength"] = df_relation_class_time["TimeLength"].astype(int)
print("Promedio:")
display(df_relation_class_time["TimeLength"].mean())
print("Min:")
display(df_relation_class_time["TimeLength"].min())
print("Max:")
display(df_relation_class_time["TimeLength"].max())
print("Tiempo de inicio")
display(df_relation_class_time.groupby("TimeStart").count())
x = df_relation_class_time.groupby("TimeStart").count()
print("Count")
display(df_relation_class_time["TimeLength"].value_counts())
print("Mediana:")
display(df_relation_class_time["TimeLength"].median())
print("Cuartiles:")
display(df_relation_class_time["TimeLength"].quantile([0, 0.25, 0.5, 0.75, 1]))
print("Var:")
display(df_relation_class_time["TimeLength"].var())
print("Std:")
display(df_relation_class_time["TimeLength"].std())

Promedio:


22.044592538624723

Min:


9

Max:


...

0.00      9.0
0.25     18.0
0.50     18.0
0.75     27.0
1.00    144.0
Name: TimeLength, dtype: float64

Var:


30.098277180969482

Std:


5.486189677815513

In [4]:
## Disponibilidad de salas en los días

for index, row in df_rooms.iterrows():
    #print(row["RoomId"], row["UnavailableDaysList"])
    if row["UnavailableDaysList"] is None:
        print(row["RoomId"], row["UnavailableDaysList"])

43 None


In [47]:
## Variabilidad de clases:

#for index, row in df_relation_class_time.iterrows():
#    print(row["ClassId"], row["TimeDays"])
display(df_relation_class_time["ClassId"].value_counts())

400     520
628     160
629     160
593     160
311     128
       ... 
527       1
1773      1
1793      1
1805      1
224       1
Name: ClassId, Length: 1852, dtype: int64

In [48]:
## Los bloques van a ser de 10 slots (50 minutos)
## Entonces la primera clase es en el slot 96 (08:00) y la ultima clase empieza en el slot 246 (20:30)
## y termina en el slot 266 (22:10)

cantidad_bloques = (256-96)/10 ## 17 bloques
duracion_bloque = 10 ## slots
dicc_bloques = {}

for i in range(1,18):
    dicc_bloques[i] = 96 + (i-1)*10

print(dicc_bloques)

conjunto_bloques = list(dicc_bloques.keys())
print(conjunto_bloques)



{1: 96, 2: 106, 3: 116, 4: 126, 5: 136, 6: 146, 7: 156, 8: 166, 9: 176, 10: 186, 11: 196, 12: 206, 13: 216, 14: 226, 15: 236, 16: 246, 17: 256}
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]


In [49]:
##Slots de tiempo y cursos que duran esos slots
print("Slots de tiempo | Cantidad de clases que duran esos slots")
dic={}
#list=[]
for index, row in df_relation_class_time.iterrows():
    #print(row["ClassId"], row["TimeLength"])
    if row["TimeLength"] not in dic.keys():
        dic[row["TimeLength"]] = [0,[]]
    if row["ClassId"] not in dic[row["TimeLength"]][1]:
        #print(row["TimeLength"],row["ClassId"], dic[row["TimeLength"]][1])
        dic[row["TimeLength"]][0]+=1
        dic[row["TimeLength"]][1].append(row["ClassId"])
c= 0
for i in dic:
    print(i, dic[i][0])
    c += dic[i][0]
print("Total",c)

Slots de tiempo | Cantidad de clases que duran esos slots
18 909
144 13
96 1
36 22
9 20
27 811
24 33
48 1
72 1
88 3
98 4
90 1
124 4
120 8
82 7
100 11
16 1
70 1
26 1
Total 1852


In [50]:
#########IGNORAR, código quizás útil para el futuro###############
#Patrones de salas según clases
diccr = {}
dic2 = {}
for index, row in df_relation_class_room.iterrows():
    #print(row["ClassId"], row["RoomId"])
    if row["ClassId"] not in diccr.keys():
        diccr[row["ClassId"]] = []
    diccr[row["ClassId"]].append(row["RoomId"])
#print(diccr)

for i in diccr:
    #print(i, diccr[i])
    tupla = tuple(diccr[i])
    if tupla not in dic2.keys():
        dic2[tupla] = []
    dic2[tupla].append(i)
#print(dic2)

c = 0

#print("Salas | Preferencias de clases por la sala")
#for i in dic2:
#    print(i, dic2[i])
#    c+=1
#print(c)

In [51]:
#Agrupar capacidad de salas, tipos de sala z (XS,S,M,L)


#print("Capacidad | Cantidad de Salas")
#display(df_rooms.groupby("Capacity").count()["RoomId"])
print("Capacidad | Cantidad de Salas | Salas")

dic_salas = {}
for index, row in df_rooms.iterrows():
    x = int(row["Capacity"])
    if x < 30:
        if "XS" not in dic_salas.keys():
            dic_salas["XS"] = [0,[]]
        dic_salas["XS"][1].append(row["RoomId"])
        dic_salas["XS"][0]+=1
    elif x == 30:
        if "S" not in dic_salas.keys():
            dic_salas["S"] = [0,[]]
        dic_salas["S"][1].append(row["RoomId"])
        dic_salas["S"][0]+=1
    elif x <= 90:
        if "M" not in dic_salas.keys():
            dic_salas["M"] = [0,[]]
        dic_salas["M"][1].append(row["RoomId"])
        dic_salas["M"][0]+=1
    else:
        if "L" not in dic_salas.keys():
            dic_salas["L"] = [0,[]]
        dic_salas["L"][1].append(row["RoomId"])
        dic_salas["L"][0]+=1
        
for i in sorted(dic_salas):
    print(i, dic_salas[i])


Capacidad | Cantidad de Salas | Salas
L [7, ['1', '3', '4', '5', '23', '24', '29']]
M [11, ['10', '11', '12', '14', '15', '16', '35', '40', '41', '42', '44']]
S [13, ['2', '9', '18', '19', '22', '28', '30', '31', '32', '34', '36', '37', '38']]
XS [13, ['6', '7', '8', '13', '17', '20', '21', '25', '26', '27', '33', '39', '43']]


In [52]:
#Agrupar capacidad de clases

print("Límite de las clases | Cantidad de clases con el límite")
display(df_classes.groupby("ClassLimit").count()["ClassId"])


Límite de las clases | Cantidad de clases con el límite


ClassLimit
10      29
100     12
11      12
12      46
120     18
128     59
129      3
13     100
130      8
14      10
15     223
18      18
180     20
20      66
21      64
22      93
24      70
25      41
26      75
28      56
30     360
32      20
36      79
38       4
40      19
42      54
43      18
44      18
48      39
50      16
52      45
60      76
64      60
75      18
90       3
Name: ClassId, dtype: int64

In [293]:

#Tiempo inicio | Frecuencia de clases en ese horario | Id de clase
dicc_bloques2 = {}
for i in dicc_bloques:
    dicc_bloques2[dicc_bloques[i]] = i
#print(dicc_bloques2)
dicct = {}
lista2 = []
df_relation_class_time["TimeStart"] = df_relation_class_time["TimeStart"].astype(int)
for index, row in df_relation_class_time.sort_values('TimeStart').iterrows():
    if row["TimeStart"] != 93:
        if str(row["TimeStart"])[-1] == "6" and dicc_bloques2[row["TimeStart"]] not in dicct.keys():
            lista2.append(dicc_bloques2[row["TimeStart"]])
            dicct[dicc_bloques2[row["TimeStart"]]] = [0,[]]
        dicct[lista2[-1]][1].append(row["ClassId"])
        dicct[lista2[-1]][0] += 1

for i in dicct:
    print(i, dicct[i][0])


1 3168
2 3074
3 3377
4 3380
5 3418
6 3407
7 3353
8 3041
9 2910
10 2812
11 2807
12 2599
13 2571
14 2541
15 2580
16 1613
17 6


In [55]:
#Cantidad de clases en una subparte
c = 0
dics = {}
dicc_s = {}
for index, row in df_subparts.iterrows():
    dicc_s[row["SubpartId"]] = row["ClassIdList"]
    
for index, col in df_configs.iterrows():
    for i in col["SubpartIdList"]:
        if len(dicc_s[i]) not in dics.keys():
            dics[len(dicc_s[i])] = [1, [], [int(col["ConfigId"])]]
            for z in dicc_s[i]:
                dics[len(dicc_s[i])][1].append(z)
                dics[len(dicc_s[i])][0]+=1
        else:
            dics[len(dicc_s[i])][2].append(int(col["ConfigId"]))
            for z in dicc_s[i]:
                dics[len(dicc_s[i])][1].append(z)
                dics[len(dicc_s[i])][0]+=1


#print("Cantidad de clases en una subparte | Cantidad de veces que se repite en las subpartes | Lista de clases | Config a la que pertenece ")
#for i in sorted(dics):
#    print(i, dics[i][0])

In [56]:
#Cantidad de subpartes de una configuración

c = 0
dics = {}
dicc_s = {}
for index, row in df_subparts.iterrows():
    dicc_s[row["SubpartId"]] = row["ClassIdList"]
    
for index, col in df_configs.iterrows():
        #print(col["ConfigId"],col["SubpartIdList"])
        if len(col["SubpartIdList"]) not in dics.keys():
            dics[len(col["SubpartIdList"])] = [0, [], [int(col["ConfigId"])], 1]
            for z in col["SubpartIdList"]:
                dics[len(col["SubpartIdList"])][0]+=1
                for w in dicc_s[z]: 
                    dics[len(col["SubpartIdList"])][1].append(w)
                    dics[len(col["SubpartIdList"])][3]+=1
        else:
            dics[len(col["SubpartIdList"])][2].append(int(col["ConfigId"]))
            for z in col["SubpartIdList"]:
                dics[len(col["SubpartIdList"])][0]+=1
                for w in dicc_s[z]: 
                    dics[len(col["SubpartIdList"])][1].append(w)
                    dics[len(col["SubpartIdList"])][3]+=1


print("Cantidad de subpartes en una config | Cantidad de veces que se repite | Lista de clases | Config a la que pertenece ")
c=0
for i in sorted(dics):
        print(i, dics[i][0])
        c+=dics[i][0]
#print(dics)



Cantidad de subpartes en una config | Cantidad de veces que se repite | Lista de clases | Config a la que pertenece 
1 32
2 258
3 45
4 16
5 45
6 96
7 49
8 24
9 207
10 70
11 33
12 144
13 13
14 14
15 105
18 18
26 26
29 29


In [298]:
import math
#Agrupar capacidad de cursos

#print("Límite de las clases | Cantidad de clases con el límite")
#display(df_classes.groupby("ClassLimit").count()["ClassId"])

dicc_cursos = {}
#display(df_courses)
for index, row in df_courses.iterrows():
    dicc_cursos[row["CourseId"]] = row["ConfigIdList"]
#print(dicc_cursos)      
dicc_configs = {}
#display(df_configs)
for index, row in df_configs.iterrows():
    dicc_configs[row["ConfigId"]] = row["SubpartIdList"]
#print(dicc_configs)
dicc_subparts = {}
#display(df_subparts)
for index, row in df_subparts.iterrows():
    dicc_subparts[row["SubpartId"]] = row["ClassIdList"]
#print(dicc_subparts)
dicc_classes = {}
#display(df_classes)
for index, row in df_classes.iterrows():
    dicc_classes[row["ClassId"]] = row["ClassLimit"]
#print(dicc_classes)

dicc_lim = {}
dicc_curso_clase = {}
for p,i in dicc_cursos.items():
    for j in i:
        if j in dicc_configs.keys():
            #print(dicc_configs[j])
            for k in dicc_configs[j]:
                if k in dicc_subparts.keys():
                    #print(dicc_subparts[k])
                    for l in dicc_subparts[k]:
                        #print(l)
                        #Agregué estas lineas para hacerme un diccionario de curso con clase#
                        if l not in dicc_curso_clase:
                            dicc_curso_clase[l] = []
                        dicc_curso_clase[l].append(p)
                        ######################################################################
                        if l in dicc_classes.keys():
                            n = int(dicc_classes[l])
                            if p not in dicc_lim.keys():
                                dicc_lim[p] = [n]
                            else:
                                dicc_lim[p].append(n)
                        
#print(dicc_lim)
#print("Id de curso | Limite de alumnos")
for i in dicc_lim.keys():
    suma = 0
    contador = 0
    for j in dicc_lim[i]:
        suma += j
        contador += 1
    promedio = suma/contador
    dicc_lim[i] = int(math.ceil(promedio/10.0))*10
   # print(i, dicc_lim[i])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [299]:
# Slot de inicio y cantidad de clases en esos slots
d = {}
for index,row in df_relation_class_time.iterrows():
    #print(row["TimeStart"])
    if int(row["TimeStart"]) not in d.keys():
        d[int(row["TimeStart"])] = [0,[]]
    d[int(row["TimeStart"])][0]+=1
    d[int(row["TimeStart"])][1].append(row["ClassId"])
#print(sorted(d.items()))
print("Slot de inicio | Conteo de clases")
#for i in sorted(d.items()):
#    print(i[0],i[1][0])


Slot de inicio | Conteo de clases


In [317]:
#Slot de inicio y cantidad de cursos en esos slots
print("Horario de inicio | Cantidad de cursos")
di = {}

for i in sorted(d.items()):
    #print(i[0],i[1][1])
    for j in set(i[1][1]):
        if i[0] not in di.keys():
            di[i[0]] = [0,[]]
        curso = dicc_curso_clase[j]
        if curso not in di[i[0]][1]:
            di[i[0]][0]+=1
            di[i[0]][1].append(curso)
print(di)

Horario de inicio | Cantidad de cursos
{93: [1, [['99']]], 96: [222, [['193'], ['211'], ['184'], ['226'], ['233'], ['182'], ['176'], ['205'], ['85'], ['160'], ['88'], ['201'], ['140'], ['210'], ['179'], ['215'], ['217'], ['18'], ['197'], ['139'], ['172'], ['108'], ['180'], ['223'], ['106'], ['196'], ['131'], ['117'], ['177'], ['109'], ['171'], ['185'], ['8'], ['55'], ['221'], ['248'], ['27'], ['243'], ['250'], ['227'], ['241'], ['79'], ['178'], ['37'], ['214'], ['206'], ['244'], ['34'], ['181'], ['237'], ['264'], ['207'], ['174'], ['222'], ['234'], ['189'], ['229'], ['188'], ['259'], ['116'], ['260'], ['191'], ['126'], ['216'], ['145'], ['15'], ['48'], ['97'], ['11'], ['124'], ['103'], ['256'], ['186'], ['219'], ['202'], ['101'], ['91'], ['200'], ['9'], ['238'], ['14'], ['132'], ['230'], ['134'], ['20'], ['198'], ['30'], ['71'], ['192'], ['228'], ['225'], ['86'], ['141'], ['46'], ['127'], ['122'], ['150'], ['194'], ['143'], ['63'], ['58'], ['224'], ['128'], ['231'], ['23'], ['236'], ['

In [170]:
## cursos más demandados por estudiantes
dic_demanda_cursos = {}
for index,row in df_students.iterrows():
    for i in row["StudentId"]:
        for j in row["CoursesList"]:
            if j not in dic_demanda_cursos.keys():
                dic_demanda_cursos[j] = 1
                #print("agregado:",j,dic_demanda_cursos[j])
            else:
                dic_demanda_cursos[j] += 1
                #print("suma +1:",j,dic_demanda_cursos[j])
#print("Id Curso | Cantidad demandada")
#for w in dic_demanda_cursos.keys():    
#    print(w,dic_demanda_cursos[w])

In [321]:
## Conteo de las salas mas utilizadas

dicc_conteo_clases_sala = {}
for index, row in df_relation_class_room.iterrows():
    if row["RoomId"] not in dicc_conteo_clases_sala.keys():
        dicc_conteo_clases_sala[row["RoomId"]] = 1
    else:
        dicc_conteo_clases_sala[row["RoomId"]] += 1
#print(dicc_conteo_clases_sala)
#print(len(dicc_conteo_clases_sala.keys()))
print("Sala | Cantidad de veces que se solicita")
#{k: v for k, v in sorted(dicc_conteo_clases_sala.items(), key=lambda item: item[1])}
#for i in sorted(dicc_conteo_clasess_sala)

Sala | Cantidad de veces que se solicita


1 ['274', '274', '274', '1', '274', '1249', '1249', '1250', '1251', '1254', '1253', '1253', '1254', '1248', '1255', '1255', '1256', '1252', '1248', '275', '1247', '1243', '1243', '277', '277', '277', '1244', '1244', '276', '276', '276', '1245', '1245', '275', '1256', '275', '1246', '1246', '1247', '1257', '1259', '1258', '1267', '261', '261', '1268', '1269', '1270', '1271', '1266', '1271', '260', '1272', '1272', '1273', '1273', '1274', '1274', '260', '1265', '1264', '262', '1258', '1259', '278', '1260', '1260', '267', '1261', '1261', '266', '1262', '1262', '264', '264', '1263', '1263', '263', '1264', '1257', '278', '279', '1242', '291', '1224', '1224', '288', '287', '1225', '1225', '291', '1226', '286', '286', '1227', '1227', '285', '285', '1228', '1226', '1223', '1223', '1222', '1217', '1217', '295', '295', '1218', '1218', '294', '294', '1219', '1219', '293', '1220', '1220', '1221', '1221', '292', '1222', '1228', '1229', '1229', '1230', '281', '281', '1238', '1238', '1239', '1239', '2

In [322]:
#Aquí se juntan los cursos que tengan en común el mismo horario de inicio y la misma cantidad de subpartes
cluster = {}
#dics subparte
#dicct horario
for i in sorted(dics):
    for j in dics[i][1]:
        for x in dicct:
            for w in dicct[x][1]:
                if j == w:
                    key = (i,x)
                    if key not in cluster.keys():
                        cluster[key] = []
                    cluster[key].append(dicc_curso_clase[j][0])
                
#print("(Cursos, cantidad de clases de subparte, horario): Clases que se adaptan")
#print(cluster)


In [323]:
## Cuantos cursos hay para cada capacidad redondeada

print("Capacidad de curso | Conteo de cursos")
dicc_cont = {}
for i in dicc_lim.values():
        if i not in dicc_cont.keys():
            dicc_cont[i] = 1
        else:
            dicc_cont[i] += 1
for i in dicc_cont.keys():
    print(i, dicc_cont[i])

Capacidad de curso | Conteo de cursos
20 21
60 20
50 35
30 128
130 6
120 3
180 2
40 46
10 1
80 8
70 2


In [324]:
## Cuantos cursos hay para cada capacidad redondeada

print("Capacidad de curso | Conteo de cursos | Cursos")
dicc_cont2 = {}
for i in dicc_lim:
    if dicc_lim[i] not in dicc_cont2.keys():
        dicc_cont2[dicc_lim[i]] = [int(i)]
    else:
        #dicc_cont2[dicc_lim[i]][0] += 1
        dicc_cont2[dicc_lim[i]].append(int(i))
#for i in dicc_cont2.keys():
#    print(i, dicc_cont2[i])

Capacidad de curso | Conteo de cursos | Cursos


In [325]:
#Relacionar cantidad de bloques para cada curso
##dicct
dicc_classes2 = {}
dicc_curso_duracion = {}
dicc_cd = {}
for index, row in df_relation_class_time.iterrows():
    dicc_classes2[row["ClassId"]] = round(row["TimeLength"]/9)


for i in dicc_curso_clase:
    if dicc_classes2[i] not in dicc_curso_duracion.keys():
        dicc_curso_duracion[dicc_classes2[i]] = []
    dicc_curso_duracion[dicc_classes2[i]].append(int(dicc_curso_clase[i][0]))

###########################################################################
#SERÁ USADO EN EL MODELO COMO PARÁMETRO
#Diccionario que relaciona la cantidad de bloques de horario con los cursos
for j in sorted(dicc_curso_duracion):
    dicc_curso_duracion[j] = set(dicc_curso_duracion[j])
#print(dicc_curso_duracion)
############################################################################

for z in dicc_curso_duracion:
    for w in dicc_curso_duracion[z]:
        if w not in dicc_cd.keys():
            dicc_cd[w] = []
        dicc_cd[w].append(z)
#print(dicc_cd)


In [326]:
cluster2 = {}
for i in cluster:
    cluster2[i] = set(cluster[i])
#print(cluster2)

In [327]:
#Cantidad de subpartes de un curso | horario de inicio del curso | capacidad de alumnos del curso = Cursos
print("Cantidad de subpartes de un curso | horario de inicio del curso | capacidad de alumnos del curso = Cursos")
cluster_cursos = {}
for i in cluster2:
    for j in cluster2[i]:
        c = 0
        for k in dicc_cont2:
            for z in dicc_cont2[k]:
                if int(j) == int(z):
                    c+=1
                    for w in dicc_cd[z]:
                        key = i[0],i[1],k,w
                        if key not in cluster_cursos:
                            cluster_cursos[key] = []
                        cluster_cursos[key].append(z)

#print(cluster_cursos)

Cantidad de subpartes de un curso | horario de inicio del curso | capacidad de alumnos del curso = Cursos


In [376]:
#Cantidad de subpartes de un curso | horario de inicio del curso | capacidad de alumnos del curso = Cursos
print("Cantidad de subpartes de un curso | horario de inicio del curso | capacidad de alumnos del curso = Cursos")
lista = []
c=0
for i in cluster_cursos:
    for j in cluster_cursos[i]:
        if j not in lista:
            lista.append(j)
            c+=1
for e,i in enumerate(cluster_cursos):
    print(e,i, cluster_cursos[i])

print("Cantidad de clusters",len(cluster_cursos))
#print("Cantidad de cursos", c)
#print("Ver cursos",sorted(lista))



Cantidad de subpartes de un curso | horario de inicio del curso | capacidad de alumnos del curso = Cursos
0 (1, 1, 60, 2) [91]
1 (1, 1, 40, 2) [72, 73, 87]
2 (1, 1, 30, 2) [35, 57, 45, 134, 153, 59, 29, 41, 166]
3 (1, 1, 30, 16) [240]
4 (1, 1, 50, 8) [241]
5 (1, 1, 50, 2) [195]
6 (1, 1, 180, 2) [19]
7 (1, 1, 20, 2) [1]
8 (1, 2, 40, 2) [72, 73, 87]
9 (1, 2, 30, 2) [35, 57, 45, 153, 59, 29, 41, 166]
10 (1, 2, 50, 2) [195]
11 (1, 2, 20, 2) [1]
12 (1, 3, 40, 2) [72, 73, 87]
13 (1, 3, 30, 2) [35, 57, 45, 134, 153, 59, 29, 41, 166, 80]
14 (1, 3, 20, 2) [3, 1]
15 (1, 3, 50, 2) [195]
16 (1, 3, 60, 5) [96]
17 (1, 4, 40, 2) [72, 73, 87]
18 (1, 4, 30, 2) [35, 57, 45, 153, 59, 29, 41, 166, 80]
19 (1, 4, 50, 2) [195]
20 (1, 4, 20, 2) [1]
21 (1, 5, 40, 2) [72, 73, 87]
22 (1, 5, 30, 2) [35, 57, 45, 153, 59, 29, 41, 166, 80]
23 (1, 5, 50, 2) [195]
24 (1, 5, 20, 2) [1]
25 (1, 5, 50, 3) [220]
26 (1, 6, 40, 2) [72, 73, 87]
27 (1, 6, 30, 2) [35, 57, 45, 153, 59, 29, 41, 166, 80]
28 (1, 6, 50, 2) [195]
29 

658 (9, 12, 60, 3) [201, 183]
659 (9, 12, 130, 3) [177]
660 (9, 12, 30, 3) [216, 209]
661 (9, 12, 40, 3) [185, 198, 181, 176, 193, 188]
662 (9, 13, 80, 3) [171, 184, 172, 197, 200]
663 (9, 13, 30, 2) [221, 223, 230, 237]
664 (9, 13, 60, 3) [201, 183]
665 (9, 13, 130, 3) [177]
666 (9, 13, 30, 3) [216, 209]
667 (9, 13, 40, 3) [185, 198, 181, 176, 193, 188]
668 (9, 14, 80, 3) [171, 184, 172, 197, 200]
669 (9, 14, 30, 2) [221, 223, 230, 237]
670 (9, 14, 60, 3) [201, 183]
671 (9, 14, 130, 3) [177]
672 (9, 14, 30, 3) [216, 209]
673 (9, 14, 40, 3) [185, 198, 181, 176, 193, 188]
674 (9, 15, 80, 3) [171, 184, 172, 197, 200]
675 (9, 15, 30, 2) [221, 223, 230, 237]
676 (9, 15, 60, 3) [201, 183]
677 (9, 15, 130, 3) [177]
678 (9, 15, 30, 3) [216, 209]
679 (9, 15, 40, 3) [185, 198, 181, 176, 193, 188]
680 (9, 16, 30, 2) [237, 221, 223, 230]
681 (10, 1, 30, 2) [249, 239, 246, 243, 242, 224]
682 (10, 1, 30, 3) [249, 239, 246, 243, 242, 224]
683 (10, 1, 40, 2) [227]
684 (10, 1, 40, 3) [227]
685 (10, 2,

In [386]:
#Conversión cluster de cursos de (s,h,c) = Cursos a (Cursos) = U (s,h,c)
cluster_cursos2 = {}
for i in cluster_cursos:
    if tuple(cluster_cursos[i]) not in cluster_cursos2.keys():
        tupla = tuple(cluster_cursos[i])
        cluster_cursos2[tupla] = []
    cluster_cursos2[tuple(cluster_cursos[i])].append(list(i))
for e,j in enumerate(cluster_cursos2):
    print(e,j,cluster_cursos2[j])
#(72, 73, 87) [[1, 1, 40, 2], [1, 2, 40, 2], [1, 3, 40, 2], [1, 4, 40, 2], [1, 5, 40, 2], [1, 6, 40, 2]]



0 (91,) [[1, 1, 60, 2]]
1 (72, 73, 87) [[1, 1, 40, 2], [1, 2, 40, 2], [1, 3, 40, 2], [1, 4, 40, 2], [1, 5, 40, 2], [1, 6, 40, 2]]
2 (35, 57, 45, 134, 153, 59, 29, 41, 166) [[1, 1, 30, 2]]
3 (240,) [[1, 1, 30, 16]]
4 (241,) [[1, 1, 50, 8]]
5 (195,) [[1, 1, 50, 2], [1, 2, 50, 2], [1, 3, 50, 2], [1, 4, 50, 2], [1, 5, 50, 2], [1, 6, 50, 2], [1, 7, 50, 2], [1, 8, 50, 2], [1, 9, 50, 2], [1, 10, 50, 2], [1, 11, 50, 2], [1, 12, 50, 2], [1, 13, 50, 2], [1, 14, 50, 2], [1, 15, 50, 2], [1, 16, 50, 2]]
6 (19,) [[1, 1, 180, 2]]
7 (1,) [[1, 1, 20, 2], [1, 2, 20, 2], [1, 4, 20, 2], [1, 5, 20, 2], [1, 6, 20, 2], [1, 7, 20, 2], [1, 8, 20, 2], [1, 9, 20, 2], [1, 10, 20, 2], [1, 11, 20, 2], [1, 12, 20, 2], [1, 13, 20, 2], [1, 14, 20, 2], [1, 15, 20, 2], [1, 16, 20, 2]]
8 (35, 57, 45, 153, 59, 29, 41, 166) [[1, 2, 30, 2]]
9 (35, 57, 45, 134, 153, 59, 29, 41, 166, 80) [[1, 3, 30, 2]]
10 (3, 1) [[1, 3, 20, 2]]
11 (96,) [[1, 3, 60, 5]]
12 (35, 57, 45, 153, 59, 29, 41, 166, 80) [[1, 4, 30, 2], [1, 5, 30, 2], 

In [422]:
dicc_perro = {}
for e, i in enumerate(cluster_cursos2.keys()):
    for k,z in enumerate(cluster_cursos.values()):
        l = list(i)
        if l == z:
            if e not in dicc_perro.keys(): 
                dicc_perro[e] = []
            dicc_perro[e].append(k)
dicc_cluster = dicc_perro

In [40]:
##################### GUROBI #######################

In [517]:
#dicc_cluster = {}
dicc_salas = {}
dicc_qnz = {}
dicc_qs = {}
dicc_lc = {}
dicc_tp = {}
dicc_qc = {}
dicc_pb = {}
#for i, j in enumerate(cluster_cursos2):
#    #print(cluster_cursos2[j])
#    for z in cluster_cursos2[j]:
#        if i not in dicc_cluster.keys():
#            dicc_cluster[i] = []
#        dicc_cluster[i].append(z[-1])
#print(dicc_cluster)
#print(dicc_bloques)

for i in dic_salas:
    dicc_salas[i] = dic_salas[i][1]
    dicc_qnz[i] = dic_salas[i][0]
    if i == "XS":
        dicc_qs[i] = 29
    elif i == "S":
        dicc_qs[i] = 30
    elif i == "M":
        dicc_qs[i] = 90
    elif i == "L":
        dicc_qs[i] = 240
    
#print(dicc_salas)
#print(dicc_qnz)
#print(dicc_qs)

#dicc_qc

for e,i in enumerate(cluster_cursos2):
    for te in cluster_cursos2[i]:
        subparte = te[0]
        #horario = list(i)[1]
        capacidad = te[2]
        tiempo = te[3]
        dicc_qc[e] = subparte
        dicc_lc[e] = capacidad
        dicc_tp[e] = tiempo
        if e not in dicc_pb:
            dicc_pb[e] = []
        dicc_pb[e].append(te[1])
        #dicc_pb[e] = horario
#print(dicc_pb)
#dicc_pb1 = {0:[1,3,5],1:[1,4,2],3:[2,5,8]}
#print(dicc_qc)
#print(dicc_lc)
#print(dicc_tp)
#print(dicc_pb)
#print(dicc_cluster)
dicc_dias2 = {}
for i in dicc_qc:
    if dicc_qc[i] == 1:
        dicc_dias2[i] = [7]
    elif dicc_qc[i] == 2:
        dicc_dias2[i] = [2,4]
    elif dicc_qc[i] == 3:
        dicc_dias2[i] = [1,3,5]
    elif dicc_qc[i] == 4:
        dicc_dias2[i] = [1,2,3,4]
    elif dicc_qc[i] == 5:
        dicc_dias2[i] = [1,2,3,4,5]
    elif dicc_qc[i] == 6:
        dicc_dias2[i] = [1,2,3,4,5,6]
    elif dicc_qc[i] > 6:
        dicc_dias2[i] = [1,2,3,4,5,6,7]
print(dicc_dias2)

{0: [7], 1: [7], 2: [7], 3: [7], 4: [7], 5: [7], 6: [7], 7: [7], 8: [7], 9: [7], 10: [7], 11: [7], 12: [7], 13: [7], 14: [7], 15: [7], 16: [7], 17: [7], 18: [7], 19: [7], 20: [7], 21: [7], 22: [7], 23: [7], 24: [7], 25: [7], 26: [2, 4], 27: [2, 4], 28: [2, 4], 29: [2, 4], 30: [2, 4], 31: [2, 4], 32: [2, 4], 33: [2, 4], 34: [2, 4], 35: [2, 4], 36: [2, 4], 37: [2, 4], 38: [2, 4], 39: [2, 4], 40: [2, 4], 41: [2, 4], 42: [2, 4], 43: [2, 4], 44: [2, 4], 45: [2, 4], 46: [2, 4], 47: [2, 4], 48: [2, 4], 49: [2, 4], 50: [2, 4], 51: [2, 4], 52: [2, 4], 53: [2, 4], 54: [2, 4], 55: [2, 4], 56: [2, 4], 57: [2, 4], 58: [2, 4], 59: [2, 4], 60: [2, 4], 61: [2, 4], 62: [2, 4], 63: [2, 4], 64: [2, 4], 65: [2, 4], 66: [2, 4], 67: [2, 4], 68: [2, 4], 69: [2, 4], 70: [2, 4], 71: [2, 4], 72: [2, 4], 73: [2, 4], 74: [2, 4], 75: [2, 4], 76: [2, 4], 77: [2, 4], 78: [2, 4], 79: [2, 4], 80: [2, 4], 81: [2, 4], 82: [2, 4], 83: [2, 4], 84: [1, 3, 5], 85: [1, 3, 5], 86: [1, 3, 5], 87: [1, 3, 5], 88: [1, 3, 5], 89: 

In [417]:
from gurobipy import *
from random   import *
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [526]:
timetabling = gp.Model("Asignacion de cursos a horarios y salas")

#indices por variable
# x: (cluster,config,bloque)
# w: (cluster,curso)
# c: (config,bloque,patron)


#diccionario cluster {id cluster: cursos dentro}
# config {id curso: }
# conjuntos/subconjuntos
########################
C = dicc_cluster
B = dicc_bloques
Z = dicc_salas
D = {1:"Lunes", 2: "Martes", 3: "Miercoles", 4: "Jueves", 5: "Viernes", 6: "Sabado", 7: "Domingo"}


# parámetros
dicc_dias = {1:"Lunes", 2: "Martes", 3: "Miercoles", 4: "Jueves", 5: "Viernes", 6: "Sabado", 7: "Domingo"}
Q_c = dicc_qc 
M = 100000000000
QN_zp = dicc_qnz
QS_z = dicc_qs
L_c = dicc_lc
h_c = 170
TP_c = dicc_tp  
PB_c = dicc_pb
#print(Q_c)
## variables

x = {} 
for c in dicc_cluster.keys():
    for d in dicc_dias.keys():
        for b in dicc_bloques.keys():
            x[c,d,b] = timetabling.addVar(vtype=gp.GRB.BINARY,name='x_'+str(c)+','+str(d)+','+str(b))
y = {}
for c in dicc_cluster.keys():
    for z in dicc_salas.keys():
        y[c,z] = timetabling.addVar(vtype=gp.GRB.BINARY,name='y_'+str(c)+','+str(z))


#Función Objetivo
N= 1 # ???:c
timetabling.setObjective(y[c,z],GRB.MINIMIZE)

#Restricciones

#1 3 5
#1 5 6
#1. Cada cluster de cursos c debe ser asignada a tipo de sala z
timetabling.addConstrs(((gp.quicksum(y[c,z] for z in Z) >= 1) for c in C), name = "R1")
###2. Cada cluster es asignado a tantas veces como clases requiera
#timetabling.addConstrs(((gp.quicksum(x[c,d,b] for d in D for b in PB_c[c]) == Q_c[c]) for c in C), name = "R2")
#3. Si hay asignación de cluster a un horario, debe ser asignado también a un tipo de sala.
timetabling.addConstrs((x[c,d,b] == y[c,z] for c in C for d in D for b in PB_c[c] for z in Z), name = "R3")
#4. El tipo de salas alcance a satisfacer la cantidad de alumnos del cluster
#timetabling.addConstrs(((y[c,z]*QS_z[z] <= x[c,d,b]*L_c[c]) for b in PB_c[c] for d in D for c in C for z in Z), name = "R4") 
##5. Limitar el tipo de salas mientras sea eficiente
timetabling.addConstrs(((h_c + QS_z[z]*y[c,z] >= x[c,d,b]*L_c[c]) for b in PB_c[c] for d in D for c in C for z in Z), name = "R5")
#6. Un cluster será asignado a tantos bloques como necesite
#timetabling.addConstrs(((gp.quicksum(x[c,d,b] for b in B) == TP_c[c]) for d in D for c in C), name = "R6")
#timetabling.addConstr((gp.quicksum(x[c,d,b] for b in range(TP_c[c]) == (gp.quicksum(x[c,d,b] for b in range(TP_c[c]) for d in D for c in C), name = "R7")
#timetabling.addConstrs(((x[c,d,b] == PB_c[c]) for b in B for d in D for c in C), name = "R8")
timetabling.optimize()
for v in timetabling.getVars():
    print('%s %g' % (v.varName, v.x))
    #print('%s %g' % (v.varName, v.y))


Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 54083 rows, 19065 columns and 108476 nonzeros
Model fingerprint: 0xefb65eaf
Variable types: 0 continuous, 19065 integer (19065 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 1.0000000
Presolve removed 54083 rows and 19065 columns
Presolve time: 0.03s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.06 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 1 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%
x_0,1,1 1
x_0,1,2 0
x_0,1,3 0
x_0,1,4 0
x_0,1,5 0
x_0,1,6 0
x_0,1,7 0
x_0,1,8 0
x_0,1,9 0
x_0,1,10 0
x_0,1,11 0
x_0,1,12 0
x_0,1,13 0
x_0,1,14 0
x_0,1,15 0
x_0,1,16 0
x_0,1,17 0
x_0,2,1 1
x_0,2,2 0
x_0,2,3 0
x_0,2,4 0
x_0,2,5 0

x_12,7,7 0
x_12,7,8 0
x_12,7,9 0
x_12,7,10 0
x_12,7,11 0
x_12,7,12 0
x_12,7,13 0
x_12,7,14 0
x_12,7,15 0
x_12,7,16 0
x_12,7,17 0
x_13,1,1 0
x_13,1,2 0
x_13,1,3 0
x_13,1,4 0
x_13,1,5 1
x_13,1,6 0
x_13,1,7 0
x_13,1,8 0
x_13,1,9 0
x_13,1,10 0
x_13,1,11 0
x_13,1,12 0
x_13,1,13 0
x_13,1,14 0
x_13,1,15 0
x_13,1,16 0
x_13,1,17 0
x_13,2,1 0
x_13,2,2 0
x_13,2,3 0
x_13,2,4 0
x_13,2,5 1
x_13,2,6 0
x_13,2,7 0
x_13,2,8 0
x_13,2,9 0
x_13,2,10 0
x_13,2,11 0
x_13,2,12 0
x_13,2,13 0
x_13,2,14 0
x_13,2,15 0
x_13,2,16 0
x_13,2,17 0
x_13,3,1 0
x_13,3,2 0
x_13,3,3 0
x_13,3,4 0
x_13,3,5 1
x_13,3,6 0
x_13,3,7 0
x_13,3,8 0
x_13,3,9 0
x_13,3,10 0
x_13,3,11 0
x_13,3,12 0
x_13,3,13 0
x_13,3,14 0
x_13,3,15 0
x_13,3,16 0
x_13,3,17 0
x_13,4,1 0
x_13,4,2 0
x_13,4,3 0
x_13,4,4 0
x_13,4,5 1
x_13,4,6 0
x_13,4,7 0
x_13,4,8 0
x_13,4,9 0
x_13,4,10 0
x_13,4,11 0
x_13,4,12 0
x_13,4,13 0
x_13,4,14 0
x_13,4,15 0
x_13,4,16 0
x_13,4,17 0
x_13,5,1 0
x_13,5,2 0
x_13,5,3 0
x_13,5,4 0
x_13,5,5 1
x_13,5,6 0
x_13,5,7 0
x_13,5,8 0
x_1

x_21,1,12 0
x_21,1,13 0
x_21,1,14 0
x_21,1,15 0
x_21,1,16 0
x_21,1,17 0
x_21,2,1 0
x_21,2,2 0
x_21,2,3 0
x_21,2,4 0
x_21,2,5 0
x_21,2,6 0
x_21,2,7 0
x_21,2,8 0
x_21,2,9 1
x_21,2,10 0
x_21,2,11 0
x_21,2,12 0
x_21,2,13 0
x_21,2,14 0
x_21,2,15 0
x_21,2,16 0
x_21,2,17 0
x_21,3,1 0
x_21,3,2 0
x_21,3,3 0
x_21,3,4 0
x_21,3,5 0
x_21,3,6 0
x_21,3,7 0
x_21,3,8 0
x_21,3,9 1
x_21,3,10 0
x_21,3,11 0
x_21,3,12 0
x_21,3,13 0
x_21,3,14 0
x_21,3,15 0
x_21,3,16 0
x_21,3,17 0
x_21,4,1 0
x_21,4,2 0
x_21,4,3 0
x_21,4,4 0
x_21,4,5 0
x_21,4,6 0
x_21,4,7 0
x_21,4,8 0
x_21,4,9 1
x_21,4,10 0
x_21,4,11 0
x_21,4,12 0
x_21,4,13 0
x_21,4,14 0
x_21,4,15 0
x_21,4,16 0
x_21,4,17 0
x_21,5,1 0
x_21,5,2 0
x_21,5,3 0
x_21,5,4 0
x_21,5,5 0
x_21,5,6 0
x_21,5,7 0
x_21,5,8 0
x_21,5,9 1
x_21,5,10 0
x_21,5,11 0
x_21,5,12 0
x_21,5,13 0
x_21,5,14 0
x_21,5,15 0
x_21,5,16 0
x_21,5,17 0
x_21,6,1 0
x_21,6,2 0
x_21,6,3 0
x_21,6,4 0
x_21,6,5 0
x_21,6,6 0
x_21,6,7 0
x_21,6,8 0
x_21,6,9 1
x_21,6,10 0
x_21,6,11 0
x_21,6,12 0
x_21,6,13 0
x

x_32,6,13 0
x_32,6,14 0
x_32,6,15 0
x_32,6,16 0
x_32,6,17 0
x_32,7,1 1
x_32,7,2 0
x_32,7,3 1
x_32,7,4 0
x_32,7,5 0
x_32,7,6 0
x_32,7,7 0
x_32,7,8 0
x_32,7,9 0
x_32,7,10 0
x_32,7,11 0
x_32,7,12 0
x_32,7,13 0
x_32,7,14 0
x_32,7,15 0
x_32,7,16 0
x_32,7,17 0
x_33,1,1 1
x_33,1,2 0
x_33,1,3 0
x_33,1,4 0
x_33,1,5 0
x_33,1,6 0
x_33,1,7 0
x_33,1,8 0
x_33,1,9 1
x_33,1,10 0
x_33,1,11 0
x_33,1,12 0
x_33,1,13 0
x_33,1,14 0
x_33,1,15 0
x_33,1,16 0
x_33,1,17 0
x_33,2,1 1
x_33,2,2 0
x_33,2,3 0
x_33,2,4 0
x_33,2,5 0
x_33,2,6 0
x_33,2,7 0
x_33,2,8 0
x_33,2,9 1
x_33,2,10 0
x_33,2,11 0
x_33,2,12 0
x_33,2,13 0
x_33,2,14 0
x_33,2,15 0
x_33,2,16 0
x_33,2,17 0
x_33,3,1 1
x_33,3,2 0
x_33,3,3 0
x_33,3,4 0
x_33,3,5 0
x_33,3,6 0
x_33,3,7 0
x_33,3,8 0
x_33,3,9 1
x_33,3,10 0
x_33,3,11 0
x_33,3,12 0
x_33,3,13 0
x_33,3,14 0
x_33,3,15 0
x_33,3,16 0
x_33,3,17 0
x_33,4,1 1
x_33,4,2 0
x_33,4,3 0
x_33,4,4 0
x_33,4,5 0
x_33,4,6 0
x_33,4,7 0
x_33,4,8 0
x_33,4,9 1
x_33,4,10 0
x_33,4,11 0
x_33,4,12 0
x_33,4,13 0
x_33,4,14 0
x

x_40,3,5 0
x_40,3,6 0
x_40,3,7 0
x_40,3,8 0
x_40,3,9 0
x_40,3,10 0
x_40,3,11 0
x_40,3,12 0
x_40,3,13 0
x_40,3,14 0
x_40,3,15 0
x_40,3,16 0
x_40,3,17 0
x_40,4,1 0
x_40,4,2 1
x_40,4,3 0
x_40,4,4 0
x_40,4,5 0
x_40,4,6 0
x_40,4,7 0
x_40,4,8 0
x_40,4,9 0
x_40,4,10 0
x_40,4,11 0
x_40,4,12 0
x_40,4,13 0
x_40,4,14 0
x_40,4,15 0
x_40,4,16 0
x_40,4,17 0
x_40,5,1 0
x_40,5,2 1
x_40,5,3 0
x_40,5,4 0
x_40,5,5 0
x_40,5,6 0
x_40,5,7 0
x_40,5,8 0
x_40,5,9 0
x_40,5,10 0
x_40,5,11 0
x_40,5,12 0
x_40,5,13 0
x_40,5,14 0
x_40,5,15 0
x_40,5,16 0
x_40,5,17 0
x_40,6,1 0
x_40,6,2 1
x_40,6,3 0
x_40,6,4 0
x_40,6,5 0
x_40,6,6 0
x_40,6,7 0
x_40,6,8 0
x_40,6,9 0
x_40,6,10 0
x_40,6,11 0
x_40,6,12 0
x_40,6,13 0
x_40,6,14 0
x_40,6,15 0
x_40,6,16 0
x_40,6,17 0
x_40,7,1 0
x_40,7,2 1
x_40,7,3 0
x_40,7,4 0
x_40,7,5 0
x_40,7,6 0
x_40,7,7 0
x_40,7,8 0
x_40,7,9 0
x_40,7,10 0
x_40,7,11 0
x_40,7,12 0
x_40,7,13 0
x_40,7,14 0
x_40,7,15 0
x_40,7,16 0
x_40,7,17 0
x_41,1,1 0
x_41,1,2 1
x_41,1,3 0
x_41,1,4 0
x_41,1,5 0
x_41,1,6 0
x_4

...

x_118,7,8 0
x_118,7,9 1
x_118,7,10 0
x_118,7,11 0
x_118,7,12 0
x_118,7,13 0
x_118,7,14 0
x_118,7,15 0
x_118,7,16 0
x_118,7,17 0
x_119,1,1 0
x_119,1,2 0
x_119,1,3 1
x_119,1,4 0
x_119,1,5 0
x_119,1,6 0
x_119,1,7 0
x_119,1,8 0
x_119,1,9 0
x_119,1,10 0
x_119,1,11 0
x_119,1,12 0
x_119,1,13 0
x_119,1,14 0
x_119,1,15 0
x_119,1,16 0
x_119,1,17 0
x_119,2,1 0
x_119,2,2 0
x_119,2,3 1
x_119,2,4 0
x_119,2,5 0
x_119,2,6 0
x_119,2,7 0
x_119,2,8 0
x_119,2,9 0
x_119,2,10 0
x_119,2,11 0
x_119,2,12 0
x_119,2,13 0
x_119,2,14 0
x_119,2,15 0
x_119,2,16 0
x_119,2,17 0
x_119,3,1 0
x_119,3,2 0
x_119,3,3 1
x_119,3,4 0
x_119,3,5 0
x_119,3,6 0
x_119,3,7 0
x_119,3,8 0
x_119,3,9 0
x_119,3,10 0
x_119,3,11 0
x_119,3,12 0
x_119,3,13 0
x_119,3,14 0
x_119,3,15 0
x_119,3,16 0
x_119,3,17 0
x_119,4,1 0
x_119,4,2 0
x_119,4,3 1
x_119,4,4 0
x_119,4,5 0
x_119,4,6 0
x_119,4,7 0
x_119,4,8 0
x_119,4,9 0
x_119,4,10 0
x_119,4,11 0
x_119,4,12 0
x_119,4,13 0
x_119,4,14 0
x_119,4,15 0
x_119,4,16 0
x_119,4,17 0
x_119,5,1 0
x_119,5,2 0


x_130,3,15 1
x_130,3,16 0
x_130,3,17 0
x_130,4,1 1
x_130,4,2 1
x_130,4,3 1
x_130,4,4 1
x_130,4,5 1
x_130,4,6 1
x_130,4,7 1
x_130,4,8 1
x_130,4,9 1
x_130,4,10 1
x_130,4,11 1
x_130,4,12 1
x_130,4,13 1
x_130,4,14 1
x_130,4,15 1
x_130,4,16 0
x_130,4,17 0
x_130,5,1 1
x_130,5,2 1
x_130,5,3 1
x_130,5,4 1
x_130,5,5 1
x_130,5,6 1
x_130,5,7 1
x_130,5,8 1
x_130,5,9 1
x_130,5,10 1
x_130,5,11 1
x_130,5,12 1
x_130,5,13 1
x_130,5,14 1
x_130,5,15 1
x_130,5,16 0
x_130,5,17 0
x_130,6,1 1
x_130,6,2 1
x_130,6,3 1
x_130,6,4 1
x_130,6,5 1
x_130,6,6 1
x_130,6,7 1
x_130,6,8 1
x_130,6,9 1
x_130,6,10 1
x_130,6,11 1
x_130,6,12 1
x_130,6,13 1
x_130,6,14 1
x_130,6,15 1
x_130,6,16 0
x_130,6,17 0
x_130,7,1 1
x_130,7,2 1
x_130,7,3 1
x_130,7,4 1
x_130,7,5 1
x_130,7,6 1
x_130,7,7 1
x_130,7,8 1
x_130,7,9 1
x_130,7,10 1
x_130,7,11 1
x_130,7,12 1
x_130,7,13 1
x_130,7,14 1
x_130,7,15 1
x_130,7,16 0
x_130,7,17 0
x_131,1,1 1
x_131,1,2 1
x_131,1,3 1
x_131,1,4 1
x_131,1,5 1
x_131,1,6 1
x_131,1,7 1
x_131,1,8 1
x_131,1,9 1
x_131

x_137,2,5 1
x_137,2,6 1
x_137,2,7 1
x_137,2,8 1
x_137,2,9 1
x_137,2,10 1
x_137,2,11 1
x_137,2,12 1
x_137,2,13 1
x_137,2,14 1
x_137,2,15 1
x_137,2,16 1
x_137,2,17 0
x_137,3,1 1
x_137,3,2 1
x_137,3,3 1
x_137,3,4 1
x_137,3,5 1
x_137,3,6 1
x_137,3,7 1
x_137,3,8 1
x_137,3,9 1
x_137,3,10 1
x_137,3,11 1
x_137,3,12 1
x_137,3,13 1
x_137,3,14 1
x_137,3,15 1
x_137,3,16 1
x_137,3,17 0
x_137,4,1 1
x_137,4,2 1
x_137,4,3 1
x_137,4,4 1
x_137,4,5 1
x_137,4,6 1
x_137,4,7 1
x_137,4,8 1
x_137,4,9 1
x_137,4,10 1
x_137,4,11 1
x_137,4,12 1
x_137,4,13 1
x_137,4,14 1
x_137,4,15 1
x_137,4,16 1
x_137,4,17 0
x_137,5,1 1
x_137,5,2 1
x_137,5,3 1
x_137,5,4 1
x_137,5,5 1
x_137,5,6 1
x_137,5,7 1
x_137,5,8 1
x_137,5,9 1
x_137,5,10 1
x_137,5,11 1
x_137,5,12 1
x_137,5,13 1
x_137,5,14 1
x_137,5,15 1
x_137,5,16 1
x_137,5,17 0
x_137,6,1 1
x_137,6,2 1
x_137,6,3 1
x_137,6,4 1
x_137,6,5 1
x_137,6,6 1
x_137,6,7 1
x_137,6,8 1
x_137,6,9 1
x_137,6,10 1
x_137,6,11 1
x_137,6,12 1
x_137,6,13 1
x_137,6,14 1
x_137,6,15 1
x_137,6,16 1
x

x_150,5,3 1
x_150,5,4 1
x_150,5,5 1
x_150,5,6 1
x_150,5,7 1
x_150,5,8 1
x_150,5,9 1
x_150,5,10 1
x_150,5,11 1
x_150,5,12 1
x_150,5,13 1
x_150,5,14 1
x_150,5,15 1
x_150,5,16 0
x_150,5,17 0
x_150,6,1 1
x_150,6,2 1
x_150,6,3 1
x_150,6,4 1
x_150,6,5 1
x_150,6,6 1
x_150,6,7 1
x_150,6,8 1
x_150,6,9 1
x_150,6,10 1
x_150,6,11 1
x_150,6,12 1
x_150,6,13 1
x_150,6,14 1
x_150,6,15 1
x_150,6,16 0
x_150,6,17 0
x_150,7,1 1
x_150,7,2 1
x_150,7,3 1
x_150,7,4 1
x_150,7,5 1
x_150,7,6 1
x_150,7,7 1
x_150,7,8 1
x_150,7,9 1
x_150,7,10 1
x_150,7,11 1
x_150,7,12 1
x_150,7,13 1
x_150,7,14 1
x_150,7,15 1
x_150,7,16 0
x_150,7,17 0
x_151,1,1 1
x_151,1,2 1
x_151,1,3 1
x_151,1,4 1
x_151,1,5 1
x_151,1,6 1
x_151,1,7 1
x_151,1,8 1
x_151,1,9 1
x_151,1,10 1
x_151,1,11 1
x_151,1,12 1
x_151,1,13 1
x_151,1,14 1
x_151,1,15 1
x_151,1,16 0
x_151,1,17 0
x_151,2,1 1
x_151,2,2 1
x_151,2,3 1
x_151,2,4 1
x_151,2,5 1
x_151,2,6 1
x_151,2,7 1
x_151,2,8 1
x_151,2,9 1
x_151,2,10 1
x_151,2,11 1
x_151,2,12 1
x_151,2,13 1
x_151,2,14 1
x_1

y_49,L 1
y_49,S 1
y_49,XS 1
y_49,M 1
y_50,L 1
y_50,S 1
y_50,XS 1
y_50,M 1
y_51,L 1
y_51,S 1
y_51,XS 1
y_51,M 1
y_52,L 1
y_52,S 1
y_52,XS 1
y_52,M 1
y_53,L 1
y_53,S 1
y_53,XS 1
y_53,M 1
y_54,L 1
y_54,S 1
y_54,XS 1
y_54,M 1
y_55,L 1
y_55,S 1
y_55,XS 1
y_55,M 1
y_56,L 1
y_56,S 1
y_56,XS 1
y_56,M 1
y_57,L 1
y_57,S 1
y_57,XS 1
y_57,M 1
y_58,L 1
y_58,S 1
y_58,XS 1
y_58,M 1
y_59,L 1
y_59,S 1
y_59,XS 1
y_59,M 1
y_60,L 1
y_60,S 1
y_60,XS 1
y_60,M 1
y_61,L 1
y_61,S 1
y_61,XS 1
y_61,M 1
y_62,L 1
y_62,S 1
y_62,XS 1
y_62,M 1
y_63,L 1
y_63,S 1
y_63,XS 1
y_63,M 1
y_64,L 1
y_64,S 1
y_64,XS 1
y_64,M 1
y_65,L 1
y_65,S 1
y_65,XS 1
y_65,M 1
y_66,L 1
y_66,S 1
y_66,XS 1
y_66,M 1
y_67,L 1
y_67,S 1
y_67,XS 1
y_67,M 1
y_68,L 1
y_68,S 1
y_68,XS 1
y_68,M 1
y_69,L 1
y_69,S 1
y_69,XS 1
y_69,M 1
y_70,L 1
y_70,S 1
y_70,XS 1
y_70,M 1
y_71,L 1
y_71,S 1
y_71,XS 1
y_71,M 1
y_72,L 1
y_72,S 1
y_72,XS 1
y_72,M 1
y_73,L 1
y_73,S 1
y_73,XS 1
y_73,M 1
y_74,L 1
y_74,S 1
y_74,XS 1
y_74,M 1
y_75,L 1
y_75,S 1
y_75,XS 1
y_75,M 1
y